In [ ]:
# %pip install langchain
# %pip install huggingface_hub
# %pip install sentence_transformers
# %pip install faiss-cpu
# %pip install unstructured
# %pip install chromadb
# %pip install Cython
# %pip install tiktoken
# %pip install unstructured[local-inference]
# %pip install qdrant-client

In [ ]:
import os
import requests
import faiss
import torch

from langchain.embeddings import HuggingFaceEmbeddings #for using HugginFace models
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS  #facebook vectorizationfrom langchain.chains.question_answering import load_qa_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings

from LangChain_chatbot_util import read_api_key

from langchain.text_splitter import CharacterTextSplitter

os.environ["HUGGINGFACEHUB_API_TOKEN"] = read_api_key()

from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain

from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate

from transformers import AutoTokenizer, AutoModel

import pandas as pd

In [ ]:
loader = DirectoryLoader('./contexts', glob="./*.txt")
 
docs = loader.load()

In [ ]:
print(docs[0].page_content)

In [ ]:
text_splitter = CharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=0
)

docs_split = text_splitter.split_documents(docs)
docs_split

In [ ]:
# Embeddings
embeddings = HuggingFaceEmbeddings()

In [ ]:
db = FAISS.from_documents(docs_split, embeddings)
query = "My teenager had a screening for autism. Where can I go for depression therapy services? "
docs_similar = db.similarity_search_with_score(query)

doc0, sim = docs_similar[0]
sim
# for i in range(4):
#     print("\n")
#     print(docs[i].page_content)

In [ ]:
def get_similarity(query, doc):
    db = FAISS.from_documents(doc, HuggingFaceEmbeddings())
    db_docs = db.similarity_search_with_score(query)
    simularity_list = []
    doc_list = []
    
    for item in db_docs:
        simularity, doc = item
        simularity_list.append(simularity)
        doc_list.append(doc)
    return simularity_list, doc_list

In [ ]:
sim, doc = get_similarity("My child keeps eating legos. Is he Autstic", docs_split)

In [ ]:
df = pd.DataFrame({"Document":doc, "Simularity": sim})
df.head()

In [ ]:
prompt = "My child keeps eating legos. Is he Autstic"
similarity_scores, indices = get_similarity(prompt, db)

# Print the similarity scores and corresponding document indices
print("Similarity Scores:", similarity_scores)
print("Document Indices:", indices)

In [ ]:
index = faiss.IndexFlatL2()